# Flyte Demo

### Running a Workflow

You can use the pre-built image:

```
export IMAGE='ghcr.io/flyteorg/flyte-demo:latest'
```

Or if you're using a local Flyte cluster and want to use an image that you
built inside it:

```
export IMAGE='flyte-demo:v1'
```

The quickest way to run a workflow is with `pyflyte run`:

```bash
pyflyte run \
    workflows/model_training.py training_workflow \
    --n_epochs 20 \
    --hyperparameters '{"in_dim": 4, "hidden_dim": 100, "out_dim": 3, "learning_rate": 0.03}'
```

This will run in locally. To run it on a Flyte cluster, simply add the `--remote`
flag:

```bash
pyflyte run \
    --remote \
    --image $IMAGE \
    workflows/model_training.py training_workflow \
    --n_epochs 20 \
    --hyperparameters '{"in_dim": 4, "hidden_dim": 100, "out_dim": 3, "learning_rate": 0.03}'
```

#### Registering Your Workflow

Once you're happy with the state of your tasks and workflows, you can register
them by first packaging them up into a portable flyte archive:

```bash
pyflyte --pkgs workflows package --force --image $IMAGE -f
```

This will create a `flyte-package.tgz` archive file that contains the serialized
tasks and workflows in this project. Then, you can register it with:

```bash
flytectl register files --project flytesnacks --domain development --archive flyte-package.tgz --version v0
```

Now we can go over to https://sandbox.union.ai/console
(or http://localhost:30080/console if you're using a local Flyte cluster) to
check out the tasks and workflows we just registered.

In [ ]:
import torch.nn as nn

from flytekit.configuration import Config
from flytekit.remote import FlyteRemote
from workflows import model_training

remote = FlyteRemote(
    config=Config.auto(config_file="./config-sandbox.yaml"),
    default_project="flytesnacks",
    default_domain="development",
)
execution = remote.execute_local_workflow(
    model_training.training_workflow,
    inputs={
        "n_epochs": 30,
        "hyperparameters": model_training.Hyperparameters(
            in_dim=4, hidden_dim=100, out_dim=3, learning_rate=0.01
        ),
    }
)
remote.generate_console_url(execution)

In [ ]:
execution = remote.wait(execution)

In [ ]:
clf = execution.outputs.get("o0", as_type=nn.Sequential)
clf

#### Scheduling Launchplans

Activate the schedule:

In [ ]:
lp_id = remote.fetch_launch_plan(name="scheduled_training_workflow").id
remote.client.update_launch_plan(lp_id, "ACTIVE")
print("activated training_launchplan")

Get the execution for the most recent schedule run.

In [ ]:
recent_executions = [
    execution
    for execution in remote.recent_executions()
    if execution.spec.launch_plan.name == "scheduled_training_workflow"
]

scheduled_execution = None
model = None
if recent_executions:
    scheduled_execution = recent_executions[0]
    scheduled_execution = remote.wait(scheduled_execution)
    model = scheduled_execution.outputs["o0"]
    model

print(model)

Now deactivate the schedule

In [ ]:
remote.client.update_launch_plan(lp_id, "INACTIVE")
print("deactivated training_launchplan")

#### `pyflyte register`

Flyte support rapid iteration during development via "fast registration" via
`pyflyte register`. This This zips up all of the source code of your Flyte 
application and bypasses the need to re-build a docker image.

```
pyflyte register --project flytesnacks --domain development --image $IMAGE workflows
```

Now go back the Flyte console and take a look at one of the workflows. You'll
see our fast-registered version under the **Recent Workflow Versions** panel.